In [12]:
from sklearn.tree import DecisionTreeClassifier
from interpret.glassbox import ExplainableBoostingClassifier
from interpret.glassbox import DecisionListClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB


clfs = {
    "CART": DecisionTreeClassifier(random_state=1234),
    "EBM": ExplainableBoostingClassifier(),
    "LR": LogisticRegression(random_state=1234),
    "GNB": GaussianNB(),
    # "DL": DecisionListClassifier(random_state=1234)
    
}

In [13]:
datasets = ['breast', 'campus', 'churn', 'climate',
            'compas', 'diabetes', 'german', 'heart',
            'stroke', 'student', 'water']

In [14]:
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import log_loss
n_datasets = len(datasets)
n_splits = 5
n_repeats = 2
rskf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=1234)

auc_scores = np.zeros((len(clfs)+1, n_datasets, n_splits * n_repeats))
loss = np.zeros((len(clfs)+1, n_datasets, n_splits * n_repeats))

In [15]:
from sklearn.base import clone 
from sklearn import metrics
import pandas as pd

import helper
import importlib
importlib.reload(helper)
from sklearn.pipeline import make_pipeline

for data_id, dataset in enumerate(datasets):
    X=pd.read_csv(f"datasets/cleaned/{dataset}_X.csv")
    X = X.drop("Unnamed: 0", axis=1)
    y = pd.read_csv(f"datasets/cleaned/{dataset}_y.csv")
    y = y.drop("Unnamed: 0", axis=1)

    features_types_df = pd.read_csv(f"datasets/cleaned/datatypes/{dataset}.csv")

    feature_inidices = list(map(int, list(features_types_df)))
    features_names = list(features_types_df.T[0])
    features_types = list(map(int, list(features_types_df.T[1])))

    preprocess = helper.select_preprocessing_for_many_feat(feature_inidices, features_types, features_names)

    for fold_id, (train, test) in enumerate(rskf.split(X, y)):
        for clf_id, clf_name in enumerate(clfs):
            clf = clone(clfs[clf_name])

            # if(clf_name == "GNB" and dataset=="compas"):
            #     clf.fit(X.iloc[train], y.iloc[train])
            #     y_preds = clf.predict(X.iloc[test])
            #     fpr, tpr, thresholds = metrics.roc_curve(y.iloc[test], y_preds)
            #     auc_scores[clf_id, data_id, fold_id] = metrics.auc(fpr, tpr)
            # else:
            clf_pipeline = make_pipeline(
                preprocess,
                clf
            )
                
            clf_pipeline.fit(X.iloc[train], y.iloc[train])
            y_preds = clf_pipeline.predict(X.iloc[test])
            fpr, tpr, thresholds = metrics.roc_curve(y.iloc[test], y_preds)
            auc_scores[clf_id, data_id, fold_id] = metrics.auc(fpr, tpr)
            loss[clf_id, data_id, fold_id] = log_loss(y.iloc[test], y_preds)

            

/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed w

In [16]:
from sklearn.base import clone 
from sklearn import metrics
import pandas as pd

import helper
import importlib
importlib.reload(helper)
from sklearn.pipeline import make_pipeline

for data_id, dataset in enumerate(datasets):
    X=pd.read_csv(f"datasets/cleaned/{dataset}_X.csv")
    X = X.drop("Unnamed: 0", axis=1)
    y = pd.read_csv(f"datasets/cleaned/{dataset}_y.csv")
    y = y.drop("Unnamed: 0", axis=1)

    features_types_df = pd.read_csv(f"datasets/cleaned/datatypes/{dataset}.csv")

    feature_inidices = list(map(int, list(features_types_df)))
    features_names = list(features_types_df.T[0])
    features_types = list(map(int, list(features_types_df.T[1])))

    preprocess = helper.select_preprocessing_for_many_feat(feature_inidices, features_types, features_names)

    for fold_id, (train, test) in enumerate(rskf.split(X, y)):

        clf_pipeline = make_pipeline(
                preprocess,
                DecisionListClassifier(random_state=1234)
            )
                
        clf_pipeline.fit(X.iloc[train], y.iloc[train])
        y_preds = clf_pipeline.predict(X.iloc[test])
        fpr, tpr, thresholds = metrics.roc_curve(y.iloc[test], y_preds)
        auc_scores[4, data_id, fold_id] = metrics.auc(fpr, tpr)
        loss[4, data_id, fold_id] = log_loss(y.iloc[test], y_preds)
        

In [17]:
auc_scores

array([[[0.91369145, 0.93694726, 0.88690476, 0.91765873, 0.95774648,
         0.89747789, 0.95103177, 0.9484127 , 0.94543651, 0.95020121],
        [0.74102564, 0.74615385, 0.83461538, 0.78817734, 0.80665025,
         0.70769231, 0.6525641 , 0.83974359, 0.75246305, 0.78694581],
        [0.67785619, 0.69595527, 0.6780116 , 0.69989173, 0.67776868,
         0.69328259, 0.68571411, 0.69144414, 0.69881515, 0.69970278],
        [0.83979592, 0.63636364, 0.7020202 , 0.59090909, 0.67171717,
         0.65918367, 0.58080808, 0.86363636, 0.61616162, 0.73737374],
        [0.55884696, 0.53869155, 0.54476471, 0.56609515, 0.57748741,
         0.53982676, 0.57585662, 0.5564976 , 0.57483112, 0.56351042],
        [0.62925926, 0.63      , 0.74037037, 0.63584906, 0.65415094,
         0.72611111, 0.66703704, 0.63703704, 0.66745283, 0.70245283],
        [0.54880952, 0.65833333, 0.65238095, 0.58452381, 0.58848921,
         0.64166667, 0.65357143, 0.65119048, 0.5452381 , 0.62769784],
        [0.69095238, 0.7071

In [18]:
auc_scores[1]

array([[0.92777596, 0.96511628, 0.95734127, 0.94047619, 0.97183099,
        0.96511628, 0.95807403, 0.97916667, 0.96428571, 0.96210597],
       [0.76282051, 0.82948718, 0.79102564, 0.8953202 , 0.91133005,
        0.85641026, 0.68076923, 0.90641026, 0.80418719, 0.78694581],
       [0.73106341, 0.73505395, 0.70966884, 0.71623318, 0.73055336,
        0.73121736, 0.71277343, 0.73572725, 0.72161298, 0.7312081 ],
       [0.65      , 0.66161616, 0.61111111, 0.66666667, 0.61111111,
        0.69489796, 0.55555556, 0.66666667, 0.55050505, 0.77777778],
       [0.57307642, 0.54069781, 0.541639  , 0.5267804 , 0.50340909,
        0.53049087, 0.56778874, 0.54755267, 0.5537125 , 0.55463133],
       [0.72277778, 0.67648148, 0.67925926, 0.74632075, 0.71801887,
        0.75833333, 0.69981481, 0.70777778, 0.69971698, 0.70471698],
       [0.64761905, 0.73809524, 0.68571429, 0.60595238, 0.63183453,
        0.66666667, 0.68690476, 0.63214286, 0.66904762, 0.6926259 ],
       [0.7647619 , 0.8247619 , 0.7074829

In [19]:
np.save('./test_results/auc/auc_results', auc_scores)
np.save('./test_results/auc/auc_losses', loss)